# Fit results

*Michele Pinamonti - INFN Sezione di Trieste*

In [1]:
f = new TFile("../ws/ATLASIT_prova_combined_ATLASIT_prova_model.root")


RooFit v3.60 -- Developed by Wouter Verkerke and David Kirkby 
                Copyright (C) 2000-2013 NIKHEF, University of California & Stanford University
                All rights reserved, please read http://roofit.sourceforge.net/license.txt

(TFile *) @0x7fb3a5ff9a78


Extract the workspace

In [2]:
w = dynamic_cast<RooWorkspace*>(f->Get("combined"));

Extract the ModelConfig from the workspace

In [3]:
mc = dynamic_cast<RooStats::ModelConfig*>(w->obj("ModelConfig"));

Get the data we want to fit

In [4]:
dataset = w->data("obsData");

Let's perform a S+B fit:
* set the initial value of the parameter of interest (POI)
* make sure it is not fixed, and it is free to float in the fit
* call the "fitTo" method

In [5]:
w->var("mu_ttH")->setVal(0);
w->var("mu_ttH")->setConstant(kFALSE);
w->pdf("simPdf")->fitTo(*dataset);

[#1] INFO:Minization -- p.d.f. provides expected number of events, including extended term in likelihood.
[#1] INFO:Minization -- RooProdPdf::getConstraints(model_ljets_HThad_5j3b) omitting term lumiConstraint as constraint term as it does not share any parameters with the other pdfs in product. To force inclusion in likelihood, add an explicit Constrain() argument for the target parameter
[#1] INFO:Minization -- createNLL: caching constraint set under name CONSTR_OF_PDF_simPdf_FOR_OBS_channelCat:obs_x_ljets_HThad_5j3b:obs_x_ljets_Mbb_ge6jge4b with 4 entries
[#1] INFO:Minization --  Including the following constraint terms in minimization: (alpha_lumiConstraint,alpha_ttXsecConstraint,alpha_JES_Scenario1_NP1Constraint,alpha_stXsecConstraint)
[#1] INFO:Fitting -- RooAddition::defaultErrorLevel(nll_simPdf_obsData_with_constr) Summation contains a RooNLLVar, using its error level
[#1] INFO:Minization -- RooMinimizer::optimizeConst: activating const optimization
RooAbsTestStatistic::initSim

From the output above, we can find the fitted values of all the parameters (of interest and nuisance) and their uncertainties.

Now, let's try to extract and print all and only the information we want, so that know could re-use it later:
* the fitted value of the POI, with its error
* the fitted values of the NPs, with their errors (what we call the "pulls" and "constraints" of the NPs)

Let's extract the POI and print the fitted value and uncertainties (separately the up and down)

In [6]:
RooRealVar *poi = (RooRealVar*)mc->GetParametersOfInterest()->first();
double mu_hat = poi->getVal();
double mu_hat_err_up = poi->getErrorHi();
double mu_hat_err_down = poi->getErrorLo();

cout << setw(25);
cout << "POI =";
cout << Form(" %+.3f",mu_hat);
cout << Form(" %+.3f",mu_hat_err_up);
cout << " /";
cout << Form(" %+.3f",mu_hat_err_down);
cout << endl;

                    POI = +1.036 +0.899 / -0.899


And now for all the NPs

In [7]:
// loop on list of nuisance parameters
for(auto np_tmp : *mc->GetNuisanceParameters()){
    RooRealVar* np = (RooRealVar*)np_tmp;
    
    string np_name = np->GetName();
    
    double np_value = np->getVal();
    double np_err_up = np->getErrorHi();
    double np_err_down = np->getErrorLo();
    
    cout << setw(25);
    cout << np_name << " =";
    cout << Form(" %+.3f",np_value);
    cout << Form(" %+.3f",np_err_up);
    cout << " /";
    cout << Form(" %+.3f",np_err_down);
    cout << endl;
}

  alpha_JES_Scenario1_NP1 = +0.203 +0.968 / -0.968
               alpha_lumi = +0.018 +0.993 / -0.993
             alpha_stXsec = +0.019 +0.993 / -0.993
             alpha_ttXsec = +0.002 +0.993 / -0.993
                    mu_tt = +1.087 +0.143 / -0.143


An alternative way (and more elegant) to store the fit results (with more information, to use later) is to use the `FitResult`

In [8]:
RooFitResult *r = w->pdf("simPdf")->fitTo(*dataset, RooFit::Save());
r->Print();

[#1] INFO:Minization -- p.d.f. provides expected number of events, including extended term in likelihood.
[#1] INFO:Minization -- createNLL picked up cached constraints from workspace with 4 entries
[#1] INFO:Minization --  Including the following constraint terms in minimization: (alpha_lumiConstraint,alpha_ttXsecConstraint,alpha_JES_Scenario1_NP1Constraint,alpha_stXsecConstraint)
[#1] INFO:Fitting -- RooAddition::defaultErrorLevel(nll_simPdf_obsData_with_constr) Summation contains a RooNLLVar, using its error level
[#1] INFO:Minization -- RooMinimizer::optimizeConst: activating const optimization
RooAbsTestStatistic::initSimMode: creating slave calculator #0 for state ljets_HThad_5j3b (5 dataset entries)
[#1] INFO:NumericIntegration -- RooRealIntegral::init(ljets_HThad_5j3b_model_Int[obs_x_ljets_HThad_5j3b]) using numeric integrator RooBinIntegrator to calculate Int(obs_x_ljets_HThad_5j3b)
RooAbsTestStatistic::initSimMode: creating slave calculator #1 for state ljets_Mbb_ge6jge4b (10

Or, as before, we might want ti extract only some of the information

In [13]:
for(auto par_tmp : r->floatParsFinal()){
    RooRealVar* par = (RooRealVar*)par_tmp;
    
    string par_name = par->GetName();
    
    double par_value = par->getVal();
    double par_err_up = par->getErrorHi();
    double par_err_down = par->getErrorLo();
    
    cout << setw(25);
    cout << par_name << " =";
    cout << Form(" %+.3f",par_value);
    cout << Form(" %+.3f",par_err_up);
    cout << " /";
    cout << Form(" %+.3f",par_err_down);
    cout << endl;
}

  alpha_JES_Scenario1_NP1 = +0.204 +0.968 / -0.968
               alpha_lumi = +0.019 +0.993 / -0.993
             alpha_stXsec = +0.019 +0.993 / -0.993
             alpha_ttXsec = +0.002 +0.993 / -0.993
                    mu_tt = +1.087 +0.143 / -0.143
                   mu_ttH = +1.036 +0.899 / -0.899


## Excercise

Let's try to produce the results of a background-only fit, and compare the output with that of the nominal S+B fit.
Hint: to perform a B-only fit, the easiest solution is to fix the POI (to what value?)